# Raport

In [17]:
import pandas as pd

In [18]:
data=pd.read_csv("train_egg_sales.csv",delimiter=';')

In [21]:
data.head()

,Date,Egg Sales
0,1993-01-01,91
1,1993-01-02,124
2,1993-01-03,112
3,1993-01-04,115
4,1993-01-05,107


In [20]:
print(data.isnull().sum()) #N/A

Date         0
Egg Sales    0
dtype: int64
